In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
year = '2017'
weeks = 4

In [3]:
url_pre = "http://ballerstein17.league.fantasy.nfl.com/research/pointsagainst?position="
url_suf = "&sort=pointsAgainst_pts&statCategory=pointsAgainst&statSeason="+year+"&statType=weekPointsAgainst&statWeek="
df = pd.DataFrame()
pos_list = ['QB','RB','WR','TE','K','DEF']
pos_nums = [1, 2, 3, 4, 7, 8]
for pos in range(len(pos_list)):
    url_base = url_pre + str(pos_nums[pos]) + url_suf
    position = pos_list[pos]
    for week in range(weeks):
        page = requests.get(url_base+str(week+1))
        soup_list = BeautifulSoup(page.content, 'html.parser').find_all('tr')
        for i in soup_list:
            row = i.find('td', class_='teamNameAndInfo')
            if row != None:
                team = list(row.find('div').children)[1]
                opp = i.find('span', class_='pointsAgainstStatId-opponent').get_text()
                pts = i.find('span', class_='pointsAgainstStatId-pts').get_text()
                df_team = pd.DataFrame({'Team_Against':[team], 'Position':[position], 'Week':(week+1), 'Year':year, 'VS':[opp], 'Points_Against':[pts], 'Game_Count':0})
                if opp != 'Bye':
                    df_team['Game_Count'] = 1
                df = pd.concat([df, df_team])

In [4]:
name = ['Miami Dolphins', 'Buffalo Bills','New England Patriots','New York Jets', 'Pittsburgh Steelers','Baltimore Ravens','Cleveland Browns','Cincinnati Bengals','Tennessee Titans','Jacksonville Jaguars','Houston Texans','Indianapolis Colts','Denver Broncos','Kansas City Chiefs','Oakland Raiders','Los Angeles Chargers','Philadelphia Eagles','Dallas Cowboys','Washington Redskins','New York Giants','Detroit Lions','Minnesota Vikings','Green Bay Packers','Chicago Bears','Atlanta Falcons','Carolina Panthers','Tampa Bay Buccaneers','New Orleans Saints','Los Angeles Rams','Seattle Seahawks','Arizona Cardinals','San Francisco 49ers','San Diego Chargers','St Louis Rams']
abbr = ['MIA','BUF','NE','NYJ','PIT','BAL','CLE','CIN','TEN','JAX','HOU','IND','DEN','KC','OAK','LAC','PHI','DAL','WAS','NYG','DET','MIN','GB','CHI','ATL','CAR','TB','NO','LA','SEA','ARI','SF','SD','STL']

for i in range(len(name)):
    df.loc[df.Team_Against.str.contains(name[i]),'Team_Against'] = abbr[i]


In [5]:
df.loc[df.VS.str.contains("@",na=False),'Home_Away'] = 'Away'
df.loc[df.Home_Away.isnull(), 'Home_Away'] = 'Home'
df.VS = df.VS.str.replace('@', '')

In [6]:
df.to_csv('data/points_against_'+year+'.csv')